<a href="https://colab.research.google.com/github/suhayb-h/Acute-Lymphoblastic-Leukemia-Classifier/blob/main/PyStart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from torch import nn, save, load
import torch
from PIL import Image
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
train = datasets.MNIST(root='data', download=True, train=True, transform=ToTensor())
#MNIST sizes are 1,28,28 (no RGB) and there are 10 classes (0-9)
dataset = DataLoader(train, 32)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [5]:
# Construct the network
class ImageClassifier(nn.Module):
  def __init__(self):
    super().__init__()
    self.model = nn.Sequential(
        nn.Conv2d(1, 32, (3,3)),
        nn.ReLU(),
        nn.Conv2d(32, 64, (3,3)),
        nn.ReLU(),
        nn.Conv2d(64,64, (3,3)),
        nn.ReLU(),
        nn.Flatten(),
        nn.Linear(64*(28-6)*(28-6), 10)
    )
  def forward(self, x):
    return self.model(x)
  
# Instantiate
clf = ImageClassifier().to('cuda')
optimizer = Adam(clf.parameters(), lr=1e-3) #lr = learning_rate
loss_fn = nn.CrossEntropyLoss()

# Training flow
if __name__ == "__main__":
  for epoch in range(10):
    for batch in dataset:
      X,y = batch
      X,y =X.to('cuda'), y.to('cuda')
      y_predict = clf(X)
      loss = loss_fn(y_predict, y)

      # Apply backprop
      optimizer.zero_grad() #0 any gradients
      loss.backward()
      optimizer.step()

    print(f"Epoch {epoch} loss is {loss.item()}")
  
  #save the model
  with open('model_state.pt', 'wb') as f:
    save(clf.state_dict(), f)

Epoch 0 loss is 0.015120006166398525
Epoch 1 loss is 0.003168223425745964
Epoch 2 loss is 0.0012812744826078415
Epoch 3 loss is 0.00013135073822923005
Epoch 4 loss is 8.641696331324056e-05
Epoch 5 loss is 9.40230893320404e-06
Epoch 6 loss is 0.00013921380741521716
Epoch 7 loss is 0.0001234216761076823
Epoch 8 loss is 0.006457239389419556
Epoch 9 loss is 5.930152838118374e-06


In [9]:
# Predict
if __name__ == "__main__":
  with open('model_state.pt', 'rb') as f: #rb = read_binary
    clf.load_state_dict(load(f))
  
  img = Image.open('image_1.jpg')
  img_tensor = ToTensor()(img).unsqueeze(0).to('cuda')

  print(torch.argmax(clf(img_tensor)) # the output is printed as tensor(0, blahblahblah). The number is the predicted class.

FileNotFoundError: ignored